In [2]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import os
from CustomFunctions import DetailedBalance

In [12]:
####### load common directories and data
time_interval = 10 #sec/frame
whichpcs = ['PC1','PC7']
basedir = 'E:/Aaron/Combined_37C_Confocal_PCA_smooth/'
datadir = basedir + 'Data_and_Figs/'
FullFrame = pd.read_csv(datadir + 'Shape_Metrics_transitionPCbins.csv', index_col=0)
centers = pd.read_csv(datadir+'PC_bin_centers.csv', index_col=0)
nbins = np.max(TotalFrame[[x for x in TotalFrame.columns.to_list() if 'bin' in x]].to_numpy())
ttot = time_interval * 180

In [10]:
### restrict data to RANDOM
treatments = ['Random']

savedir = basedir + 'random/'
if not os.path.exists(savedir):
    os.makedirs(savedir)

#restrict dataframe to only random experiments
TotalFrame = FullFrame[FullFrame.Treatment=='Random'].copy()

In [13]:
### restrict data to PARANITROBLEBBISTATIN
treatments = ['DMSO','Para-Nitro-Blebbistatin']

savedir = basedir + 'Para-Nitro-Blebbistatin/'
if not os.path.exists(savedir):
    os.makedirs(savedir)

#limit data to the Para-Nitro-Blebbistatin experiments
TotalFrame = FullFrame[FullFrame.Experiment == 'Drug'].copy()
dates = [20240624,20240626,20240701,20241125,20241126,20241127]
TotalFrame = TotalFrame[TotalFrame.Date.isin(dates)]
TotalFrame['Treatment'] = pd.Categorical(TotalFrame.Treatment.to_list(), categories=treatments, ordered=True)

In [15]:
### restrict data to CK666
treatments = ['DMSO','CK666']

savedir = basedir + 'CK666/'
if not os.path.exists(savedir):
    os.makedirs(savedir)

#limit data to the CK666 experiments
TotalFrame = FullFrame[FullFrame.Experiment == 'Drug'].copy()
dates = [20240610,20240617,20240620,20241205,20241209]
TotalFrame = TotalFrame[TotalFrame.Date.isin(dates)]
TotalFrame['Treatment'] = pd.Categorical(TotalFrame.Treatment.to_list(), categories=treatments, ordered=True)

In [16]:
###################

if __name__ ==  '__main__':
    ########### get raw transitions and pairs ###########
    rawtrans, rawpairs = DetailedBalance.get_raw_cgps_trajectories(
            TotalFrame, #pandas dataframe with all of the cgps binned data
            whichpcs, #which two PCs to use in the cgps [x,y]
            time_interval, #real time between datapoints
            savedir, #where to save the aggregated trajectories
            )


    ########### interpolate all transitions so that only individual transitions are made ###########
    transdf_sep, transpairsdf_sep = DetailedBalance.get_interpolated_cgps_trajectories(
            TotalFrame, #pandas dataframe with all of the cgps binned data
            whichpcs, #which two PCs to use in the cgps [x,y]
            time_interval, #real time between datapoints
            savedir, #where to save the aggregated trajectories
            )
    
    
    ############## get the counts of cells leaving 
    trans_rate_df_sep = DetailedBalance.aggregate_transition_counts(
            transdf_sep, #transdf_sep from get_interpolated_cgps_trajectories
            whichpcs, #which two PCs to use in the cgps [x,y]
            savedir, #where to save the aggregated counts
            nbins, #how many bins in the x and y cgps axes
            )

    ############## BOOTSTRAP MANY TRAJECTORIES ##########
    bstrans, bsint, bspairs, bsframe_sep_full = DetailedBalance.get_bootstrapped_cgps_trajectories(
            rawpairs, #raw transition pairs from get_raw_cgps_trajectories
            whichpcs, #which two PCs to use in the cgps [x,y]
            time_interval, #real time between datapoints
            savedir, #where to save the aggregated counts
            nbins, #how many bins in the x and y cgps axes
            ttot, #set the total bootstrap time
            bsiter = 3000, #number of times to bootstrap
            )


    ############# open average bootstrapped currents ###################
    bsfield_sep = DetailedBalance.get_avg_current_error(
            bsframe_sep_full, #transition rates in the cgps from get_bootstrapped_cgps_trajectories
            whichpcs, #which two PCs to use in the cgps [x,y]
            savedir, #where to save the aggregated counts
            nbins, #how many bins in the x and y cgps axes
            )
    

Aggregated transitions
Finished interpolating trajectories
168161.58979173415
190520.4769209914
Finished finding transition rates
Starting CK666 trajectory 0-300
Starting CK666 trajectory 301-600
Starting CK666 trajectory 601-900
Starting CK666 trajectory 901-1200
Starting CK666 trajectory 1201-1500
Starting CK666 trajectory 1501-1800
Starting CK666 trajectory 1801-2100
Starting CK666 trajectory 2101-2400
Starting CK666 trajectory 2401-2700
Starting CK666 trajectory 2701-3000
Finished CK666 rates 0, starting 0-300
Finished CK666 rates 301, starting 301-600
Finished CK666 rates 601, starting 601-900
Finished CK666 rates 901, starting 901-1200
Finished CK666 rates 1201, starting 1201-1500
Finished CK666 rates 1501, starting 1501-1800
Finished CK666 rates 1801, starting 1801-2100
Finished CK666 rates 2101, starting 2101-2400
Finished CK666 rates 2401, starting 2401-2700
Finished CK666 rates 2701, starting 2701-3000
Starting DMSO trajectory 0-300
Starting DMSO trajectory 301-600
Starting D

In [ ]:
############# create all CGPSs

binlist = [i for i in TotalFrame.columns.to_list() if 'bin' in i]
for a in binlist:
    for b in binlist:
        bin1 = a.split('bin')[0]
        bin2 = b.split('bin')[0]
        print(bin1, bin2)
        if a == b:
            continue